In [9]:
words = open('names.txt','r').read().splitlines()
import torch
import torch.nn.functional as F
torch.set_default_device('cuda')

In [10]:

chars = ['.'] + sorted(list(set(''.join(words))))
chars_bi = []
for i in chars:
    for j in chars:
        ch_ = i+j
        chars_bi.append(''.join(ch_))

In [11]:
stoi_bi = {s:i for i,s in enumerate(chars_bi)}
itos_bi = {i:s for s,i in stoi_bi.items()}
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}

In [14]:

xs, ys = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi_bi[ch1+ch2]
        ix3 = stoi[ch3]
        xs.append(ix1)
        ys.append(ix3)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ',num)

number of examples:  196113


In [19]:

# g = torch.Generator().manual_seed(2147483647)
xenc = F.one_hot(xs, num_classes=729).float()
W = torch.randn((729, 27), requires_grad=True)

In [23]:
for k in range(100):
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(num), ys].log().mean()
    print(loss.item())
    
    W.grad = None
    loss.backward()

    W.data += -30 * W.grad

2.28286075592041
2.282288074493408
2.2817182540893555
2.281151294708252
2.2805874347686768
2.28002667427063
2.2794690132141113
2.278914213180542
2.278362274169922
2.277813196182251
2.2772672176361084
2.276723861694336
2.2761833667755127
2.2756457328796387
2.275110960006714
2.274578809738159
2.2740492820739746
2.2735228538513184
2.272998809814453
2.272477626800537
2.271959066390991
2.2714428901672363
2.2709295749664307
2.270418882369995
2.2699105739593506
2.269404888153076
2.268901824951172
2.2684013843536377
2.2679030895233154
2.2674076557159424
2.266914129257202
2.2664237022399902
2.265935182571411
2.265449285507202
2.2649660110473633
2.2644846439361572
2.2640058994293213
2.2635293006896973
2.2630550861358643
2.2625832557678223
2.2621138095855713
2.261646270751953
2.261181116104126
2.26071834564209
2.2602579593658447
2.2597992420196533
2.259342908859253
2.2588887214660645
2.258436918258667
2.2579872608184814
2.2575392723083496
2.257093667984009
2.256649971008301
2.256208896636963
2.25

In [24]:
for i in range(5):
    out = []
    ix = 0

    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=729).float()
        logits = xenc @ W
        counts = logits.exp()
        p = counts / counts.sum(1, keepdims=True)

        ix = torch.multinomial(p, num_samples=1, replacement=True).item()
        out.append(itos[ix])
        if ix % 27 == 0:
            break
    print(''.join(out))

heleisanaanadupalanizevariaiwidaquravadanazanerevinazonyavisaxhelehazelaneshelichizayoauselesaevalolianafacazelelaradaramenaumevihantelafqugotaleishalyocostayziparanadepokaaamaisubrxzilimanoladayfanabempelesijubronaiharabaiwilodabaramaarhanamaezhazoridadiquwemoshaliyimyefamanoluwbeevaledvryoooluvimasaruxxmelu.
habelarisbewaitisarisugasomiaadetamailenazataimevqu.
luamelanisazaanadroriamaxmiowzalaaremy.
oshalidiwaliasevimimiisnadaristhaysuvadzaladalalyezywzoelanoledeyadelavadoxamasmabrheadayaivemarocevikalchabemosadow.
heliregrebeyyeprileberpamalaiharevemesavvemiayjevimerhelilavidaalanemadonaxrheatarvadeoprevihanalidemelemalevicowrevirivaluoremcaunivevauvelmisatocaledivitanoualemademarelomezarotyanbhazyahivwamalyabooproezamelaviloluxalyeveluxnanedryalhaadgiyochadlizarynyomzoreayemaryhaliurevivarsalelelxnakaraalanyasadaiyaiquosaydqxaamidelolymeloluvyemamahalelelalazamadadayolshamehahalevanoxmazuvxruonarasemadevarphoashanavaowainabejamelorohalamadamanvanadazanoruvapavalisuas.
